In [1]:
!pip install dash==0.21.1 
!pip install dash-renderer==0.13.0  
!pip install dash-html-components==0.11.0
!pip install dash-core-components==0.23.0  
!pip install plotly --upgrade
!pip install jupyter-dash
!pip install pandas

  Using cached dash-0.21.1-py3-none-any.whl

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dash-bootstrap-components 1.0.2 requires dash>=2.0.0, but you have dash 0.21.1 which is incompatible.



  Attempting uninstall: dash
    Found existing installation: dash 2.0.0
    Uninstalling dash-2.0.0:
      Successfully uninstalled dash-2.0.0
  Using cached dash_html_components-0.11.0-py3-none-any.whl
  Attempting uninstall: dash-html-components
    Found existing installation: dash-html-components 2.0.0
    Uninstalling dash-html-components-2.0.0:
      Successfully uninstalled dash-html-components-2.0.0
  Using cached dash_core_components-0.23.0-py3-none-any.whl
  Attempting uninstall: dash-core-components
    Found existing installation: dash-core-components 2.0.0
    Uninstalling dash-core-components-2.0.0:
      Successfully uninstalled dash-core-components-2.0.0


In [ ]:
# Importation des bibliotheques
import dash
import pandas as pd
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pickle

#Recuperation des donnees 
data =  pd.read_csv('C:/Users/TerminalPc/Desktop/Stat-Eco/visualisation de donnees/airline.csv')

# Une fonction qui permet d'afficher le dataframe sous forme d'une table

def generate_table(dataframe, max_rows=20):
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in dataframe.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))
        ])
    ])

#Regrouper par mois et par compagnie aerienne les retards  mateorologique
meteo = data.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
#Nettoyage des valeurs nulles en les remplacant par 0
meteo['WeatherDelay']=meteo['WeatherDelay'].fillna(0)
#Regrouper par mois et par compagnie aerienne les retards des transporteurs
transp = data.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
#Nettoyage des valeurs nulles en les remplacant par 0
transp['CarrierDelay']=transp['CarrierDelay'].fillna(0)
#Regrouper par mois et par compagnie aerienne les retards du systeme nationnal aerien
nsa = data.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
#Nettoyage des valeurs nulles en les remplacant par 0
nsa['NASDelay']=nsa['NASDelay'].fillna(0)
#Regrouper par mois et par compagnie aerienne les retards de la securite
security = data.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
#Nettoyage des valeurs nulles en les remplacant par 0
security['SecurityDelay']=security['SecurityDelay'].fillna(0)
#Regrouper par mois et par compagnie aerienne les retards des avions
rd_avions = data.groupby(['Month','Reporting_Airline'])['ArrDelay'].mean().reset_index()
#Nettoyage des valeurs nulles en les remplacant par 0
rd_avions['ArrDelay']=rd_avions['ArrDelay'].fillna(0)

# Initialisation de l'application
app = dash.Dash(__name__,supress_calback_exceptions=True)

colors = {
    'background': '#F0FFFF',
    'text': '#4169E1'
}

app.layout = html.Div(style={'backgroundColor': colors['background']},children=[
                 html.H1("Un dashboard pour analyser les retards des vols aux Etats Unis", style = {'font-size': '50px','color':'#808090','border': '5px solid Silver','border-radius': '30px', 'box-shadow': '1px 1px 0px black'}),
                 html.Div([
                     dcc.Dropdown(id='year',multi=False,
                                     options= [{'label':x, 'value':x} for x in data['Year'].unique()],
                                     placeholder= 'Selectionner un pays',
                                     ),
                 ],style={'width':'500px'}),
                 html.Div([
                     html.H2('Les moyennes par mois et par annee',
                            style = {'textAlign': 'center' ,'color': colors['text']}
                            ),
                     html.P(id='rtd_carrier', children= generate_table(transp),
                            style={'display': 'inline-block','vertical-align': 'middle','left': 50, 'color':colors['text'], 'fontSize':15,'border': '3px solid #808097','border-radius': '20px'}
                            ),
                     html.Div(dcc.Graph(id='bar',figure=px.scatter(transp,x='Month',y='CarrierDelay',hover_name="Reporting_Airline",log_x=True,color="Reporting_Airline"))),
                     html.P(id='rtd_weather', children= generate_table(meteo),
                            style={'margin':25,'display': 'inline-block','vertical-align': 'middle', 'color':colors['text'], 'fontSize':15,'border': '3px solid #808097','border-radius': '20px'}
                            ),
                     html.Div(dcc.Graph(id='barM',figure=px.bar(meteo,x='Month',y='WeatherDelay', color="Reporting_Airline", barmode="group"))),
                     html.P(id='rtd_nsa', children= generate_table(nsa),
                            style={'display': 'inline-block','vertical-align': 'middle','color':colors['text'], 'fontSize':15,'border': '3px solid #808097','border-radius': '20px'}
                            ),
                     html.Div(dcc.Graph(id='barN',figure=px.histogram(nsa,x='Month',y='NASDelay', color="Reporting_Airline", hover_name="Reporting_Airline"))),
                     html.P(id='rtd_security', children= generate_table(security),
                            style={'margin':25,'display': 'inline-block','vertical-align': 'middle','color':colors['text'], 'fontSize':15,'border': '3px solid #808097','border-radius': '20px'}
                            ),
                     html.Div(dcc.Graph(id='barS',figure=px.bar(security,x='Month',y='SecurityDelay', color="Reporting_Airline", barmode="group"))),
                     html.P(id='rtd_avion', children= generate_table(rd_avions),
                            style={'display': 'inline-block','vertical-align': 'middle','color':colors['text'], 'fontSize':15,'border': '3px solid #808097','border-radius': '20px'}
                            ),
                     html.Div(dcc.Graph(id='barA',figure=px.bar(rd_avions,x='Month',y='ArrDelay',color="Reporting_Airline",barmode="group"))),
                 ]),

             ])

#Callback Pour les transporteurs
@app.callback(
     Output('rtd_carrier','children'),
     [Input('year','value')],
)

def update_value(year):
    if not year:
        return f"Retard des transporteurs par mois"
    else:
        transp = data[data['Year']==year].groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
        return generate_table(transp)

#Callback pour le graphique des retards des transporteurs
@app.callback(
     Output('bar','figure'),
     [Input('year','value')],
)

def update_value(year):
    if not year:
        return px.scatter(transp,x='Month',y='CarrierDelay',hover_name="Reporting_Airline",log_x=True,color="Reporting_Airline",size_max=60,size="CArrierDelay")
    else:
        transp = data[data['Year']==year].groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
        graphTransp = px.scatter(transp,x='Month',y='CarrierDelay',hover_name="Reporting_Airline",log_x=True,color="Reporting_Airline",size_max=60,size="CArrierDelay")
        return graphTransp 
    
#Callback Pour le retard de la meteorologie
@app.callback(
   Output('rtd_weather','children'),
    [Input('year','value')],
)

def meteo_value(year):
    if not year:
        return f"Retard meteorologique par mois"
    else:
        meteo = data[data['Year']==year].groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
        return generate_table(meteo)

#Callback pour le graphique des retards meteorologique
@app.callback(
   Output('barM','figure'),
    [Input('year','value')],
)

def meteo_value(year):
    if not year:
        return px.bar(meteo,x='Month',y='WeatherDelay',color="Reporting_Airline", barmode="group")
    else:
        meteo = data[data['Year']==year].groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
        graphM = px.bar(meteo,x='Month',y='WeatherDelay',color="Reporting_Airline", barmode="group")
        return graphM 

#Callback Pour les retards du système aérien national    
@app.callback(
    Output('rtd_nsa','children'),
     [Input('year','value')],
)

def nsa_value(year):
    if not year:
        return f"Retard du système aérien national par mois"
    else:
        nsa = data[data['Year']==year].groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
        return generate_table(nsa)                   

#Callback pour le graphique des retards du NSA
@app.callback(
    Output('barN','figure'),
     [Input('year','value')],
)

def nsa_value(year):
    if not year:
        return  px.sunburst(nsa,path=['Month','NASDelay'], color="Reporting_Airline", hover_data=nsa.columns,values="NASDelay")
    else:
        nsa = data[data['Year']==year].groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
        graphNSA = px.sunburst(nsa,path=['Month','NASDelay'], color="Reporting_Airline", hover_data=nsa.columns,values="NASDelay")
        return graphNSA     
    
#Callback Pour les retards de la securite
@app.callback(
    Output('rtd_security','children'),
     [Input('year','value')],
)

def security_value(year):
    if not year:
        return f"Retard de Securite par mois"
    else:
        security = data[data['Year']==year].groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
        return generate_table(security)
    
#Callbck pour le graphique des retards de la securite
@app.callback(
    Output('barS','figure'),
     [Input('year','value')],
)

def security_value(year):
    if not year:
        return px.bar(security,x='Month',y='SecurityDelay',color="Reporting_Airline", barmode="group")
    else:
        security = data[data['Year']==year].groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
        graphS = px.bar(security,x='Month',y='SecurityDelay',color="Reporting_Airline", barmode="group")
        return graphS          
        
#Callback Pour les retards des Avions                     
@app.callback(
    Output('rtd_avion','children'),
    [Input('year','value')]
)

def avion_value(year):
    if not year:
        return f"Retard des Avions par mois"
    else:
        rd_avions = data[data['Year']==year].groupby(['Month','Reporting_Airline'])['ArrDelay'].mean().reset_index()
        return generate_table(rd_avions)
    
#Callback pour le graphique des retards des avions
@app.callback(
    Output('barA','figure'),
    [Input('year','value')]
)

def avion_value(year):
    if not year:
        return px.bar(rd_avions,x='Month',y='ArrDelay',color="Reporting_Airline", barmode="group")
    else:
        rd_avions = data[data['Year']==year].groupby(['Month','Reporting_Airline'])['ArrDelay'].mean().reset_index()
        graphA = px.bar(rd_avions,x='Month',y='ArrDelay',color="Reporting_Airline", barmode="group")
        return graphA   
    
if __name__ == '__main__':
    app.run_server(port=8002)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8002/ (Press CTRL+C to quit)
